In [1]:
!pip install --upgrade transformers datasets accelerate peft

In [2]:
# This step is needed only on Apple Metal
!pip uninstall bitsandbytes -y

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # Replace with the appropriate Llama 3.1 model name

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# Set up LoRA configuration
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout probability
    bias="none",  # Don't add bias to the LoRA adapters
    target_modules=['down_proj', 'gate_proj', 'o_proj', 'v_proj', 'up_proj', 'q_proj', 'k_proj'],
    task_type="CAUSAL_LM",
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# Check if pad token exists, add it if missing
if tokenizer.pad_token is None:
   print(f'Added pad_token {tokenizer.eos_token}')
   tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

# Update the model's token embeddings to accommodate the new pad token
model.resize_token_embeddings(len(tokenizer))

Added pad_token <|eot_id|>


Embedding(128256, 4096)

In [5]:
import torch
print(f'MPS available: {torch.backends.mps.is_available()}')

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = model.to(device)

MPS available: True


Prepare data

In [6]:
from datasets import load_dataset

dataset = load_dataset("ruslanmv/ai-medical-chatbot", split='all')
dataset

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 256916
})

In [7]:
# Inspired by https://github.com/meta-llama/llama-recipes/blob/main/src/llama_recipes/datasets/alpaca_dataset.py
# See also https://crfm.stanford.edu/2023/03/13/alpaca.html

from copy import deepcopy

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


IGNORE_INDEX = -100 # The default setting in CrossEntropyLoss

# Tokenize the dataset
def tokenize_function(example):
    if example.get("Patient", "") == "":
        prompt = PROMPT_DICT["prompt_no_input"].format(instruction=example['Description'])
    else:
        prompt = PROMPT_DICT["prompt_input"].format(instruction=example['Description'], input=example['Patient'])

    # In decoder-only transformers, the task is to predict the next token.
    # The sequence is fed as input_ids, and labels are the next token in the sequence.
    # Suppose the sequence is: ["The", "capital", "of", "France", "is", "Paris"]
    # Input (input_ids): ["The", "capital", "of", "France", "is", "Paris"]
    # Labels: ["capital", "of", "France", "is", "Paris", "<EOS>"] (Shifted version of the input)

    formatted_example = prompt + example['Doctor']

    encoded_prompt = torch.tensor(
            tokenizer.encode(prompt), dtype=torch.int64
        )
        
    example = tokenizer.encode(formatted_example)
    example.append(tokenizer.eos_token_id)
    example = torch.tensor(
        example, dtype=torch.int64
    )
    labels = deepcopy(example)
    labels[: len(encoded_prompt)] = -1
    example_mask = example.ge(0)
    label_mask = labels.ge(0)
    example[~example_mask] = 0
    labels[~label_mask] = IGNORE_INDEX  # Avoid calculating loss on prompt predictions

    return {
        "input_ids": example.tolist(),
        "labels": labels.tolist(),
        "attention_mask": example_mask.tolist(),
    }


tokenized_dataset = dataset.map(tokenize_function, remove_columns=['Description', 'Patient', 'Doctor'], batched=False)
tokenized_dataset

Dataset({
    features: ['input_ids', 'labels', 'attention_mask'],
    num_rows: 256916
})

In [8]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(10))

Training

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding='longest',
)

In [10]:
import math

# Define the perplexity metric function
def compute_metrics(eval_pred):
    print(eval_pred)
    logits, labels = eval_pred

    # Convert logits and labels from NumPy arrays to PyTorch tensors
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)
    
    # Shift the labels so that they're aligned with the next token prediction
    labels = labels[:, 1:].reshape(-1)
    logits = logits[:, :-1].reshape(-1, logits.shape[-1])
    
    # Compute cross-entropy loss
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(logits, labels)

    # Compute perplexity from loss
    perplexity = math.exp(loss.item()) if loss.item() < 100 else float("inf")
    
    return {"perplexity": perplexity}

In [11]:
from transformers import TrainingArguments, Trainer
from torch.optim import AdamW

training_args = TrainingArguments(
    output_dir="./llama_lora_finetuned",
    eval_strategy="steps",
    eval_steps = 1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    save_steps=1000,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    warmup_steps=10,
    load_best_model_at_end=True,
)

optimizer = AdamW(model.parameters(), lr=5e-5)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    optimizers=(optimizer, None),  # No need for a scheduler here
    compute_metrics=compute_metrics
)


In [12]:
trainer.train()

  0%|          | 0/25691 [00:00<?, ?it/s]

{'loss': 2.7792, 'grad_norm': 2757.18115234375, 'learning_rate': 5e-06, 'epoch': 0.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 3.1393580436706543, 'eval_perplexity': 13.806453627430567, 'eval_runtime': 28.4721, 'eval_samples_per_second': 0.351, 'eval_steps_per_second': 0.351, 'epoch': 0.0}
{'loss': 2.7648, 'grad_norm': 5642.0068359375, 'learning_rate': 1e-05, 'epoch': 0.0}


  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 3.1372902393341064, 'eval_perplexity': 13.791070144522328, 'eval_runtime': 62.8129, 'eval_samples_per_second': 0.159, 'eval_steps_per_second': 0.159, 'epoch': 0.0}
{'loss': 2.8702, 'grad_norm': 2993.759765625, 'learning_rate': 2e-05, 'epoch': 0.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 3.1335065364837646, 'eval_perplexity': 13.758264918925999, 'eval_runtime': 67.9742, 'eval_samples_per_second': 0.147, 'eval_steps_per_second': 0.147, 'epoch': 0.0}


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("./llama3_lora_model_finetuned")
tokenizer.save_pretrained("./llama3_lora_tokenizer_finetuned")